In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:

import os
import shutil
import random

def split_test_data(source_dir, val_dir, test_dir, val_split=0.2):
    # Ensure destination directories exist
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    # List all class directories in source_dir
    class_names = os.listdir(source_dir)
    for class_name in class_names:
        class_source_dir = os.path.join(source_dir, class_name)
        class_val_dir = os.path.join(val_dir, class_name)
        class_test_dir = os.path.join(test_dir, class_name)

        if not os.path.exists(class_val_dir):
            os.makedirs(class_val_dir)
        if not os.path.exists(class_test_dir):
            os.makedirs(class_test_dir)

        # List all image files in the class directory
        image_files = os.listdir(class_source_dir)
        num_images = len(image_files)
        num_val_images = int(num_images * val_split)
        val_images = random.sample(image_files, num_val_images)

        # Move images to validation or test directories
        for image_file in image_files:
            src_path = os.path.join(class_source_dir, image_file)
            if image_file in val_images:
                dest_path = os.path.join(class_val_dir, image_file)
            else:
                dest_path = os.path.join(class_test_dir, image_file)
            shutil.copy(src_path, dest_path)

        print(f"Class '{class_name}': Moved {num_val_images} images to validation set.")

# Define paths
source_dir = '/content/drive/MyDrive/플젝....../test_data'
validation_dir = '/content/drive/MyDrive/플젝....../validation'
final_test_dir = '/content/drive/MyDrive/플젝....../final_test'

# Run split
split_test_data(source_dir, validation_dir, final_test_dir)


Class 'SAD': Moved 260 images to validation set.
Class 'surprise': Moved 224 images to validation set.
Class 'joy': Moved 316 images to validation set.
Class 'angry': Moved 266 images to validation set.


In [ ]:
#라벨링
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Define paths
train_data_folder = '/content/drive/MyDrive/플젝....../train_data'
validation_data_folder = '/content/drive/MyDrive/플젝....../validation'
image_size = (128, 128)  # Adjust to your input size

def load_images_and_labels(data_folder):
    images = []
    labels = []

    # List of class names and create a class to index mapping
    class_names = sorted(os.listdir(data_folder))
    class_dict = {name: idx for idx, name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(data_folder, class_name)
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            image = load_img(img_path, target_size=image_size)  # Load image and resize
            image = img_to_array(image)  # Convert image to array
            images.append(image)
            labels.append(class_dict[class_name])  # Append corresponding label

    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load and preprocess training data
train_images, train_labels = load_images_and_labels(train_data_folder)

# Load and preprocess validation data
val_images, val_labels = load_images_and_labels(validation_data_folder)

# Optional: Split training data into train and validation sets (if needed)
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, shuffle=True)

# Confirm the shapes of the arrays
print(f"Training data shape: {x_train.shape}, Labels shape: {y_train.shape}")
print(f"Validation data shape: {x_val.shape}, Labels shape: {y_val.shape}")
print(f"Validation data shape: {val_images.shape}, Labels shape: {val_labels.shape}")


In [ ]:
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# parameters
batch_size = 32
num_epochs = 1000
image_shape = (128,128)
validation_split = .2
num_classes = 4
patience = 50
base_path = '/content/drive/MyDrive/얼굴인식/최종데이터/model/'

train_dir = '/content/drive/MyDrive/얼굴인식/최종데이터/final_test_data의 사본'
validation_dir = '/content/drive/MyDrive/얼굴인식/최종데이터/validation_data의 사본'

# Create ImageDataGenerator instances for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,  # Resize images to 128x128
    batch_size=batch_size,
    class_mode='categorical'  # For multi-class classification
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical'
)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
# 베이스 모델의 레이어를 고정
for layer in base_model.layers[:-20]:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # 드롭아웃 추가
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)

model=Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])
model.summary()

log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
model_names = base_path + 'resnet.{epoch:02d}-{val_accuracy:.2f}.keras'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)

callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbacks,
    verbose=1
)

for layer in base_model.layers:
    layer.trainable = True

# 학습률을 낮춰서 재학습
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

# 모델 학습 (재학습)
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,  # 추가 에포크
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=callbacks,
    verbose=1
)


NameError: name 'ImageDataGenerator' is not defined

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np


# Paths to test data directory
test_dir = '/content/drive/MyDrive/플젝....../최종데이터/final_test_data'

# Create ImageDataGenerator instance for test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=64,
    class_mode='categorical',
    shuffle=False  # Important to set shuffle=False for correct predictions mapping
)

Found 2670 images belonging to 4 classes.


In [ ]:

print(f'Training samples: {train_generator.samples}')
print(f'Validation samples: {validation_generator.samples}')

In [ ]:
from tensorflow.keras.models import load_model

best_model_path = '/content/drive/MyDrive/플젝....../model2/resnet.28-0.92.keras'
best_model = load_model(best_model_path)

batch_size=64
# Evaluate the best model
test_loss, test_accuracy, test_precision, test_recall = best_model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size,
    verbose=1
)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")


41/41 ━━━━━━━━━━━━━━━━━━━━ 1030s 26s/step - accuracy: 0.7716 - loss: 0.5308 - precision_3: 0.7959 - recall_3: 0.7360
Test Loss: 0.4839664101600647
Test Accuracy: 0.8094512224197388
Test Precision: 0.8396381735801697
Test Recall: 0.7782012224197388


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Predict the classes of the test data
test_generator.reset()
predictions = model.predict(test_generator, steps=test_generator.samples // batch_size + 1, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# True classes
true_classes = test_generator.classes

# Class labels
class_labels = list(test_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)
